In [52]:
import cv2
import numpy as np
import math
import scipy.stats as st
from skimage.exposure import rescale_intensity
import hw1
import cv2
import helpers
import random


In [10]:
img = cv2.imread('images/cactus.jpg' ,cv2.IMREAD_COLOR)


In [11]:
image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [12]:
# Kernels
sobel_x = np.array(
    [
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ], dtype=np.float32
)
sobel_y = np.array(
    [
        [1, 2, 1],
        [0, 0, 0],
        [-1, -2, -1]
    ], dtype=np.float32
)



In [13]:
# Convolute kernels
G_x = hw1.convolution(image_gray, sobel_x, sobel_x.shape[0], sobel_x.shape[0], add= False, in_place=False)
G_y = hw1.convolution(image_gray, sobel_y, sobel_y.shape[0], sobel_y.shape[0], add=False, in_place=False)



In [28]:
cv2.imshow("Img",(G_x/8))
cv2.waitKey(0)


27

In [18]:
cv2.imshow("Img",G_y)
cv2.waitKey(0)


27

In [31]:
G_x = G_x/8.
G_y = G_y/8.
# Magnitude
GG = np.sqrt(G_x * G_x + G_y * G_y)

# Orientation
GA = np.arctan(np.divide(G_y, G_x))

/var/folders/yh/s9_d6k391270nzt5kng8nsfw0000gn/T/ipykernel_36168/3769021697.py:7: RuntimeWarning: divide by zero encountered in true_divide
  GA = np.arctan(np.divide(G_y, G_x))
/var/folders/yh/s9_d6k391270nzt5kng8nsfw0000gn/T/ipykernel_36168/3769021697.py:7: RuntimeWarning: invalid value encountered in true_divide
  GA = np.arctan(np.divide(G_y, G_x))


In [35]:
GG = hw1.rescale_int(GG)
#cv2.imshow("Img",GG)
cv2.imwrite("GG3.png",GG)
#cv2.waitKey(0)


27

In [71]:
# Orientation
GA = np.zeros_like(G_x, dtype="float32")
GA[G_x > 0] = np.arctan(G_y[G_x > 0]/ G_x[G_x > 0]) # Avoid division by 0

In [68]:
np.min(GA), np.max(GA)

(0.0, 1.5668747)

In [61]:
GA = hw1.rescale_int(GA*255.)
print(np.mean(GA))

40.962650646678426


In [72]:
GA = GA*255.
#GA = hw1.rescale_int(GA)
#cv2.imshow("Img",GA)
cv2.imwrite("GA3.png",GA)
#cv2.waitKey(0)

True

In [70]:
GA_scaled = (GA - np.min(GA))/(np.max(GA) - np.min(GA))
cv2.imwrite("GA3.png",GA_scaled)


True

In [66]:
sobel = image_gray + GG + GA
sobel = hw1.rescale_int(sobel)
cv2.imshow("Img",sobel)

cv2.waitKey(0)

27

In [25]:
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

-1

In [2]:
img = cv2.imread('images/yosemite.png' ,cv2.IMREAD_COLOR)


In [3]:
sigma = 1.0
alpha = 5.0

In [4]:
#second_diff = hw1.second_deriv_image(img, sigma, in_place=False, add=False)
# 2D Laplacian of Gaussian (LoG)
kernel_log = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])



In [5]:
# Convolution
diff_im = hw1.convolution(img, kernel_log, kernel_log.shape[0], kernel_log.shape[0], False, in_place=False)
blurred_img = hw1.gaussian_blur_image(diff_im, sigma)


In [ ]:
# Avoid negative values
img_non_negative = np.clip(blurred_img, 0., 255.)


In [7]:
second_diff = blurred_img

In [6]:
print(np.min(blurred_img))
print(np.max(blurred_img))


-56.10098598350014
34.66877278759053


In [8]:
#second_diff = np.array(second_diff - 128.)
#second_diff = np.clip(second_diff, 0., 255., second_diff)
aux = alpha * second_diff

#sharp_img = img - np.clip(aux, 0., 255.)
sharp_img = img - aux

In [9]:
print(np.min(sharp_img))
print(np.max(sharp_img))


-104.17000162505673
504.5049299175007


In [11]:
sharp_img = hw1.rescale_int(sharp_img)


In [12]:
cv2.imshow("Img",sharp_img)

cv2.waitKey(0)


27

In [13]:
cv2.imshow("Img",cv2.cvtColor(sharp_img, cv2.COLOR_BGR2GRAY))

cv2.waitKey(0)

27

In [53]:
# k-means
image = cv2.imread('images/flowers.png', cv2.IMREAD_COLOR)

num_clusters = 4


In [54]:
# Define criteria = ( type, max_iter = 100 , epsilon = 1.0 )
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 30.0)

# Set flags
flags = cv2.KMEANS_PP_CENTERS

# Reshape for cv2 function
image = (image).astype(np.float32)
Z = image.reshape(-1, 3)


In [44]:
# Apply KMeans
compactness, labels, center = cv2.kmeans(Z, num_clusters, None, criteria, 10, flags)


In [45]:
centers = np.uint8(center*255.)
res = centers[labels.flatten()]
image_k = res.reshape((image.shape))



In [46]:
cv2.imshow('Image', image_k)
cv2.waitKey(0)

27

In [55]:
centroids = np.zeros((num_clusters,3))
for i in range(num_clusters):
    centroids[i, 0] = random.randint(0,255)
    centroids[i, 1] = random.randint(0, 255)
    centroids[i, 2] = random.randint(0, 255)

In [56]:
from scipy.spatial.distance import cdist

out = cdist(Z, centroids, metric='cityblock')

In [57]:
out

array([[192., 246., 195., 189.],
       [193., 245., 196., 190.],
       [191., 247., 194., 188.],
       ...,
       [236., 300., 193., 135.],
       [278., 378., 255., 139.],
       [287., 397., 274., 158.]])

In [47]:
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)


-1